In [1]:
banned_ent_types = {"ORGANIZATION", "EVENT", "GPE", "LOC"}
banned_pos = ["PUNCT", "SPACE"]
reflexive_deps = ["expl:poss", "expl:pv", "iobj", "obj"]
root_forms = ["ROOT", "advcl", "acl", "cop", 'csubj', "conj", "parataxis", "ccomp", "ccomp:pmod"]

reflexive_short_to_long_form = {
    "mi-": "îmi",
    "ți-": "îți",
    "și-": "își",
    "v-": "vă",
    "s-": "se",
    "ne-": "ne",
    "te-": "te",
    "i-": "îi"
}

ud_to_dex = {
        "VERB": "V",
        "AUX": "V",
        "PART": "I",
        "NOUN": "M",
        "PROPN": "SP",
        "PRON": "P",
        "DET": "P",
        "SCONJ": "I",
        "CCONJ": "I",
        "NUM": "P",
        "INTJ": "I",
        "ADV": "I",
        "ADP": "I",
        "ADJ": "A",
        "X": "V"
   }
end_of_phrase = ["!", "?", ".", "\n"]

json_archive = "utils_json.zip"
json_archive_url = f"https://github.com/PetruTH/nlp_lic/releases/download/Resources/{json_archive}"
UNIDENTIFIED_TOKEN = "unidentified"
MAPARE_PATH = "util/forme_morfologice.json"
ALL_INFLECTED_FORMS_PATH = "util/inflected_form_lexemeId_inflectionId.json"
WORD_TO_ID_POS_PATH = "util/word_id_pos.json"
ID_TO_WORD_POS_PATH = "util/id_word_pos.json"
ID_TO_INFLECTED_FORMS_PATH = "util/wordId_inflected_forms.json"
print("gata..")
from spacy.tokens import Token


gata..


In [2]:
from data_loader import load_jsons
from util_data import (
    UNIDENTIFIED_TOKEN,
    ud_to_dex,
    banned_pos,
    banned_ent_types,
    reflexive_deps,
    reflexive_short_to_long_form
)

mapare, all_inflected_forms, word_to_id_pos, id_to_word_pos, id_to_inflected_forms, entry_lexeme, tree_entry, relation, synonyms, context = load_jsons()


def get_all_forms_worker(token: Token) -> [int]:
    """
    thiw will extract every word having inflected form == token.text
    """
    token_text = token.text
    if "-" in token.text:
        token_text = token_text.replace("-", "")

    all_inflected_words_found = all_inflected_forms.find_all_inflected_forms_double_verification(
                token_text, token_text.lower()
            )

    if all_inflected_words_found == UNIDENTIFIED_TOKEN:
        return []

    words_prel = []
    only_one_word = [word['lexemeId'] for word in all_inflected_words_found]

    if len(set(only_one_word)) == 1:
        words_prel.append(str(only_one_word[0]))
    for word in all_inflected_words_found:
        pos_found = mapare.find_dexonline_pos_id(word['inflectionId'])
        """
            mapare.mapping['DEXONLINE_MORPH']: ["morph dexonline", "pos dexonline"],
            this will help for mapping spacy pos to dexonline pos
            mapping spacy pos with dexonline pos
            looking after an id found from dexonline
        """

        if ud_to_dex[token.pos_] == pos_found:
            if str(word['lexemeId']) not in words_prel:
                words_prel.append(str(word['lexemeId']))

        elif ud_to_dex[token.pos_] == "M" and pos_found == "F":
            if str(word['lexemeId']) not in words_prel:
                words_prel.append(str(word['lexemeId']))

        elif ud_to_dex[token.pos_] == "M" and pos_found == "N":
            if str(word['lexemeId']) not in words_prel:
                words_prel.append(str(word['lexemeId']))

    words_prel.sort(key=lambda x: int(x))

    return words_prel


def get_all_forms(token: Token) -> [{str, str}]:
    """
        This function will return all the inflected forms for a certain token given as a parameter.
        It will search for that token in dexonline database and it will find the lexemeId.
        Based on get_all_forms_worker, it will choose the word from the list returned that
        has lemma like the first form found in dexonline database. After, that,
        based on that lexemeId, it will return all inflected forms found with the same lexemeId (a list of
        dictionaries containig words form and morphological details also from dexonline database)
    """
    words_prel = get_all_forms_worker(token)
    token_text = token.text

    if len(words_prel) > 1:
        for element in words_prel:
            if id_to_word_pos.find_id_to_word_pos_form(element) == token.lemma_:
                id = element

    elif len(words_prel) == 1:
        id = words_prel[0]

    elif len(words_prel) == 0:
        words_found = word_to_id_pos.find_word_id_pos_double_verification(token.lemma_, token_text)

        if words_found != UNIDENTIFIED_TOKEN:
            words_prel = [str(x['id']) for x in words_found]
            id = words_prel[0]
        else:
            return []

    result = id_to_inflected_forms.find_id_to_inflected_forms(id)

    return result


def validate_token(token: Token) -> bool:
    """
        Function that validates if a token can be found in dexonline database.
        It will exclude words that describe names or places, organizations, etc.
    """
    if "-" in token.text:
        return True
    if token.pos_ in banned_pos:
        return False
    if token.lang_ != "ro":
        return False
    if not token.text.isalpha():
        return False
    if token.ent_type_ in banned_ent_types:
        return False
    return True


def get_wanted_form(token: Token, pos_finder: str, person: str, number: str) -> str:
    """
       This function will return the morph form wanted by pos_finder, person and number
    """
    all_morph = get_all_forms(token)
    for wanted_form in all_morph:
        if pos_finder in wanted_form['pos'] and person in wanted_form['pos'] and number in wanted_form['pos']:
            return wanted_form['form']
    return "UNKNOWN"


def verify_word_at_certain_pos(token: Token, pos_verifier: str) -> bool:
    """
    verifiy if a token is contains a specified string in its part of speech
    for example this function will return true if a verb has this description from dexonline
    as its pos "Verb, Indicativ, perfect simplu, persoana I, singular" and pos_verifier parameter
    is "perfect simplu" or "persoana I", etc
    """
    all_morph = get_all_forms(token)
    for wanted_form in all_morph:
        if token.text == wanted_form['form']:
            for pos in pos_verifier:
                if pos in wanted_form['pos']:
                    return True


def is_composed_subj(token: Token) -> bool:
    # extra step to verify if there is a composed subject (like 'eu cu tine mergem')
    if not token.pos_ == "VERB" and not token.pos_ == "AUX":
        if len(list(token.children)):
            for t in token.children:
                if t.text not in ["m", "te", "s"]:
                    return 1
        return 0


def get_right_person_and_number(token: Token) -> (str, str):
    """
        This function will get the person and number data from token.morph
        and will convert these into dexonline database format information
        in order to select right form of verb.
    """
    # extract correct person and number for a phrase
    person = token.morph.get("Person", ['3'])
    number = token.morph.get("Number", ['Sing'])

    if is_composed_subj(token):
        number = ["Plur"]

    # formatting number and person to be recognized dexonline json
    actual_number = "plural" if number == ["Plur"] else "singular"

    if person == ['1']:
        actual_person = "I"
    elif person == ['2']:
        actual_person = "II"
    elif person == ['3']:
        actual_person = "III"

    return actual_number, actual_person


def forme_reflexive_verifier(token: Token) -> str:
    """
        This function will map short reflexive forms into long ones
        using data from reflexive_deps from util_data.py
    """
    word_added = token.text
    if token.dep_ in reflexive_deps:
        case_condition = token.morph.get("Case", ["dummy"])[0] in ["Dat", "Acc"]
        variant_condition = token.morph.get("Variant", ["dummy"])[0] == "Short"
        if case_condition and variant_condition:
            word_added = reflexive_short_to_long_form[token.text]

    return word_added



from spacy.tokens import Token

Token.set_extension("forms_", method=get_all_forms, force=True)
Token.set_extension("is_valid", method=validate_token, force=True)


def find_lexeme_ids(inflected_forms: [str]) -> [str]:
    possible_lexeme_ids = []

    if inflected_forms != ["UNKNOWN"]:
        for inflected_form in inflected_forms:
            if inflected_form.get("lexemeId") not in possible_lexeme_ids:
                possible_lexeme_ids.append(inflected_form.get("lexemeId"))
  
    
    return possible_lexeme_ids

def find_inflection_possibilites(token: Token, inflected_forms: [str], pos_wanted: str) -> [str]:
    inflection_possibilites = []

    if inflected_forms != ["UNKNOWN"]:
        for inflected_form in inflected_forms:
            inflectionId = mapare.find_dexonline_pos_id(inflected_form["inflectionId"])
            
            inflected_form_id = str(inflected_form["inflectionId"])

            if inflectionId == pos_wanted and inflected_form_id not in inflection_possibilites:
                inflection_possibilites.append(str(inflected_form["inflectionId"]))
            elif inflectionId in ["VT", "V"] and pos_wanted in ["V", "VT"] and inflected_form_id not in inflection_possibilites:
                inflection_possibilites.append(str(inflected_form["inflectionId"]))
            elif inflectionId in ["M", "F", "N"] and pos_wanted in ["M", "F", "N"] and inflected_form_id not in inflection_possibilites:
                inflection_possibilites.append(str(inflected_form["inflectionId"]))
            elif token.dep_ in ["ROOT", "nmod"] and inflected_form_id not in inflection_possibilites:
                inflection_possibilites.append(str(inflected_form["inflectionId"]))

    return inflection_possibilites

def find_matching_lexemeIds(possible_lexeme_ids: [str], pos_wanted: str) -> [str]:
    lexeme_ids = [] 
   
    for lexemeId in possible_lexeme_ids:
        variant = id_to_word_pos.find_id_to_word_pos(lexemeId)
        if variant['pos'] == pos_wanted:
            lexeme_ids.append(lexemeId)
        elif variant['pos'] in ["VT", "V", "AUX"] and pos_wanted in ["V", "VT", "AUX"]:
            lexeme_ids.append(lexemeId)
        elif variant['pos'] in ["M", "F", "N"] and pos_wanted in ["M", "F", "N"]:
            lexeme_ids.append(lexemeId)
    
    return lexeme_ids

def find_entryIds(lexeme_ids: str) -> str:
    entry_ids = []
    for lexemeId in lexeme_ids:
        all_entries = entry_lexeme.find_entry_lexeme(lexemeId)
        if all_entries != ["no entry"]:
            for entry in all_entries:
                entry_ids.append(entry)

    return entry_ids

def find_treeIds(entry_ids: str) -> str:
    tree_ids = []
    for entryId in entry_ids:
        tree_entries = tree_entry.find_tree_entry(entryId)
        if tree_entries != ["no entry tree"]:
            for treeId in tree_entries:
                tree_ids.append(treeId)
    
    return tree_ids

def find_meaningIds(tree_ids: str) -> str:
    meaning_ids = []

    for treeId in tree_ids:
        all_meaningIds = relation.find_relation(str(treeId))
        if all_meaningIds != ["no relation"]:
            for meaningId in all_meaningIds:
                meaning_ids.append(meaningId)

    return meaning_ids

 [INFO] 2024-06-21 18:57:58,507 root:177 - Start loading needed data in memory!
 [INFO] 2024-06-21 18:57:58,511 data_loader:189 - Mapare file loaded.


 [INFO] 2024-06-21 18:58:03,890 data_loader:191 - All inflected forms file loaded.
 [INFO] 2024-06-21 18:58:04,808 data_loader:193 - Mapping word to id and pos file loaded.
 [INFO] 2024-06-21 18:58:05,749 data_loader:195 - Mapping word id to word and pos file loaded.
 [INFO] 2024-06-21 18:58:08,827 data_loader:197 - Mapping id to inflected forms file loaded.
 [INFO] 2024-06-21 18:58:09,898 data_loader:199 - Mapping entry id to lexeme id file loaded.
 [INFO] 2024-06-21 18:58:10,099 data_loader:201 - Mapping tree id to entry id file loaded.
 [INFO] 2024-06-21 18:58:10,145 data_loader:203 - Mapping meaning id to tree id file loaded.
 [INFO] 2024-06-21 18:58:10,283 data_loader:205 - Mapping synonyms file loaded.
 [INFO] 2024-06-21 18:58:10,320 data_loader:207 - Mapping contexts file loaded.
 [INFO] 2024-06-21 18:58:10,321 root:209 - The data from jsons files is now loaded in memory!


In [3]:

import re
from spacy.tokens import Token
from json_creator import incarcare_eficienta
from util_data import reflexive_short_to_long_form

def synonyms_builder_step1(token: Token, pos_wanted: str)  -> ([str], [str]):
    token_text = re.sub('[^a-zA-ZăâîșțĂÂÎȘȚ]', '', token.text.lower())
    inflected_forms = all_inflected_forms.find_all_inflected_forms(token_text)
    try:
        inflection_possibilities = find_inflection_possibilites(token, inflected_forms, pos_wanted)
    except:
        inflection_possibilities = []
    possible_lexeme_ids = find_lexeme_ids(inflected_forms)
    lexeme_ids = find_matching_lexemeIds(possible_lexeme_ids, pos_wanted)
    entry_ids = find_entryIds(lexeme_ids)
    tree_ids = find_treeIds(entry_ids)
    meaning_ids = find_meaningIds(tree_ids)

    if len(inflection_possibilities) > 1:
        inflection_possibilities =  inflection_filter(token=token, inflection_possibilities=inflection_possibilities)
    return tree_ids, inflection_possibilities, meaning_ids

def synonyms_builder_step2(meaning_ids, tree_id_forced, token):
    candidate_synonyms_base_form = []
    token_text = re.sub('[^a-zA-ZăâîșțĂÂÎȘȚ]', '', token.text.lower())
    # print(tree_id_forced, "tree ids forced step2")

    for meaningId in meaning_ids:
        
        possible_synonyms = synonyms.find_synonyms(meaningId)
        tree_ids_verifier = [syn[0] for syn in possible_synonyms]
        # print(tree_ids_verifier, f"tree ids for meaning id {meaningId}", int(tree_id_forced[0]) in tree_ids_verifier)

        # print(possible_synonyms)
        if possible_synonyms != ["no synonyms"]:
        
            for synonym in possible_synonyms:
                # aici adauga si treeid si mai departe selectezi doar form si cand le sugerezi dai pe cele cu matching treeid (chosencontext de mai jos)
                # undo ca inainte stergi syn_treeId
                syn_to_add = re.sub('[^a-zA-ZăâîșțĂÂÎȘȚ ]', '', synonym[1]).split(" ")
               
                for syn in syn_to_add:
                    syn_to_add_helper = all_inflected_forms.find_all_inflected_forms(syn, unidentified={"lexemeId": "UNKNOWN"})
                    if syn_to_add == ["UNKOWN"]:
                        break

                    syn_tuple = (syn, syn_to_add_helper[0].get("lexemeId", "dummy"))
                    if syn_tuple not in candidate_synonyms_base_form and syn_tuple[0] != token_text:  
                        if int(tree_id_forced[0]) in tree_ids_verifier:      
                            candidate_synonyms_base_form.append(syn_tuple)

    candidate_synonyms_base_form = [syn for i, syn in enumerate(candidate_synonyms_base_form) if i == 0 or syn[1] != candidate_synonyms_base_form[i-1][1]]
    # print(candidate_synonyms_base_form, "!!!!")
    return candidate_synonyms_base_form

def is_valid_for_syn(token: Token) -> bool:
    if token.pos_ == "PUNCT":
        return False
    if "aux" in token.dep_:
        return False
    if not token.text.isalpha():
        return False
    return True



def get_all_forms_worker(token: Token) -> [int]:
    """
    thiw will extract every word having inflected form == token.text
    """
    token_text = token.text
    if "-" in token.text:
        token_text = token_text.replace("-", "")

    all_inflected_words_found = all_inflected_forms.find_all_inflected_forms_double_verification(
                token_text, token.lemma_
            )

    if all_inflected_words_found[0] == UNIDENTIFIED_TOKEN:
        return [-1]

    words_prel = []
    
    only_one_word = [word['lexemeId'] for word in all_inflected_words_found]

    if len(set(only_one_word)) == 1:
        words_prel.append(str(only_one_word[0]))
    for word in all_inflected_words_found:
        pos_found = mapare.find_dexonline_pos_id(word['inflectionId'])
        """
            mapare.mapping['DEXONLINE_MORPH']: ["morph dexonline", "pos dexonline"],
            this will help for mapping spacy pos to dexonline pos
            mapping spacy pos with dexonline pos
            looking after an id found from dexonline
        """

        if ud_to_dex[token.pos_] == pos_found:
            if str(word['lexemeId']) not in words_prel:
                words_prel.append(str(word['lexemeId']))

        elif ud_to_dex[token.pos_] == "M" and pos_found == "F":
            if str(word['lexemeId']) not in words_prel:
                words_prel.append(str(word['lexemeId']))

        elif ud_to_dex[token.pos_] == "M" and pos_found == "N":
            if str(word['lexemeId']) not in words_prel:
                words_prel.append(str(word['lexemeId']))

    words_prel.sort(key=lambda x: int(x))

    return words_prel


def get_all_forms(token: Token) -> [{str, str}]:
    """
        This function will return all the inflected forms for a certain token given as a parameter.
        It will search for that token in dexonline database and it will find the lexemeId.
        Based on get_all_forms_worker, it will choose the word from the list returned that
        has lemma like the first form found in dexonline database. After, that,
        based on that lexemeId, it will return all inflected forms found with the same lexemeId (a list of
        dictionaries containig words form and morphological details also from dexonline database)
    """
    id = -1
    words_prel = get_all_forms_worker(token)
    token_text = token.text

    if len(words_prel) > 1:
        for element in words_prel:
            if id_to_word_pos.find_id_to_word_pos_form(element) == token.lemma_:
                id = element

    elif len(words_prel) == 1:
        if words_prel[0] == -1:
            return []
        else:
            id = words_prel[0]

    elif len(words_prel) == 0:
        # print(token, "!!!", words_prel)
        words_found = word_to_id_pos.find_word_id_pos_double_verification(token.lemma_, token_text)

        if words_found != UNIDENTIFIED_TOKEN:
            words_prel = [str(x['id']) for x in words_found]
            id = words_prel[0]
        else:
            return []

    result = id_to_inflected_forms.find_id_to_inflected_forms(id)

    return result

def get_wanted_form(token: Token, pos_finder: str, person: str, number: str) -> str:
    """
       This function will return the morph form wanted by pos_finder, person and number
    """
    all_morph = get_all_forms(token)
    for wanted_form in all_morph:
        if pos_finder in wanted_form['pos'] and person in wanted_form['pos'] and number in wanted_form['pos']:
            return wanted_form['form']
    return "UNKNOWN"

def get_synonyms(token: Token, tree_id_forced = []) -> [str]:
    if is_valid_for_syn(token):
        pos_found = ud_to_dex[token.pos_]
        tree_ids, inflection_possibilites, meaning_ids = synonyms_builder_step1(token, pos_found)

        candidate_synonyms_base_form = synonyms_builder_step2(meaning_ids, tree_id_forced, token)

        synonyms_found = []
        # print(candidate_synonyms_base_form, "candidate la sinonime")
        for syn in candidate_synonyms_base_form:
            inflected_forms_syn = id_to_inflected_forms.find_id_to_inflected_forms(str(syn[1]))

            for inflectionId in inflection_possibilites:
                inflection = mapare.find_dexonline_pos_detail(str(inflectionId))

                for pos_syn in inflected_forms_syn:
                    pos_found_on_syn = pos_syn.get("pos")
                    form_found_on_syn = pos_syn.get("form")
                    if pos_found_on_syn == inflection:
                            if form_found_on_syn not in synonyms_found:
                                synonyms_found.append(form_found_on_syn)
                                
       
        return synonyms_found
    else:
        return []

def force_plural_noun(token):
    associated_tokens = token.subtree
    for token in associated_tokens:
        if token.pos_ in ["DET", "NUM", "PRON"]:
            if token.morph.get("Number", ["dummy"])[0] == "Plur":
                return True
    return False

def force_person_and_number_verb(token):
    number, person = "_", "_"
    inf = False
    subtree = token.subtree
    
    for t in subtree:
        if t == token:
            break
        if t.dep_ == "nsubj":
            number = t.morph.get("Number", ["Sing"])[0]

            if number == "Plur":
                number = "plural"
            else:
                number = "singular"
                        
            if t.pos_ == "NOUN":
                person = "a III-a"
            elif t.pos_ == "PRON":
                person = t.morph.get("Person", ["dummy"])[0]

                if person == "1":
                    person = "I"
                elif person == "2":
                    person = "a II-a"
                elif person == "3":
                    person = "a III-a"
        elif t.dep_ == "mark":
            inf = True
    
            
    return inf, number, person

def get_verb_tense(token):
    mood = token.morph.get("Mood", ["Ind"])[0]
    tense = token.morph.get("Tense", ["dummy"])[0]
    verbform = token.morph.get("VerbForm", ["dummy"])[0]

    if tense == "Imp" or tense == "Pres" and mood == "Indicativ" and verbform == "Inf":
        tense = "imperfect"
    elif tense == "Pres":
        tense = "prezent"
    elif tense == "Past":
        tense = "perfect simplu"
    elif tense == "Pqp":
        tense = "mai mult ca perfect"
    return tense

def build_inflection_for_verb(token, inflection_dex_details):
    mood = "Indicativ" if token.morph.get("Mood", ["Ind"])[0] == "Ind" else "None"

    inf, number, person = force_person_and_number_verb(token)

    tense = get_verb_tense(token)

    if token.dep_ == "ccomp" or inf == True:
        found_dex_pos = "Verb, Infinitiv prezent"
    else:
        found_dex_pos = f"Verb, {mood}, {tense}, persoana {person}, {number}"

    if found_dex_pos == inflection_dex_details:
        return True
    else:
        found_dex_pos = f"Verb, {mood}, prezent, persoana {person}, {number}"
        if found_dex_pos == inflection_dex_details:
            return True
    return False

def get_case_for_noun(token):
    case = token.morph.get("Case", ["Acc", "Nom"])
    if "Acc" in case[0] or "Nom" in case[0]:
        case = "Acc, Nom"

    if case == "Acc, Nom":
        case = "Nominativ-Acuzativ"
    elif case == "Dat, Gen":
        case = "Genitiv-Dativ"
    else:
        case = "Vocativ"
    return case

def get_definite(token):
    definite = token.morph.get("Definite", ["dummy"])[0]
    if definite == "Ind":
        definite = "nearticulat"
    else:
        definite = "articulat"
    return definite

def get_number(token):
    number = token.morph.get("Number", ["Sing"])[0]
    
    if force_plural_noun(token):
        number = "plural"
    elif number == "Sing":
        number = "singular"
    else:
        number = "plural"
    return number

def get_gender(token):
    gender = token.morph.get("Gender", ["dummy"])[0]
    if gender == "Masc":
        gender = "masculin"
    else: 
        gender = "feminin"

def get_case_for_pron(token):
    case = token.morph.get("Poss", ["No"])[0]
    if case == "Yes":
        case = "Genitiv-Dativ"
    else:
        case = "Nominativ-Acuzativ"
    return case

def build_inflection_for_noun(token, inflection_dex_details):
    case = get_case_for_noun(token)
    definite = get_definite(token)
    number = get_number(token)
    gender = get_gender(token)

    if token.pos_ == "NOUN":
        dex_pos = "Substantiv"
    elif token.pos_ == "ADJ":
        dex_pos = "Adjectiv"

    found_dex_pos = f"{dex_pos} {gender}, {case}, {number}, {definite}"
    if found_dex_pos == inflection_dex_details:
        return True
    elif f"{dex_pos} neutru, {case}, {number}, {definite}" == inflection_dex_details:
        return True
    return False
    
def build_inflection_for_pron(token, inflection_dex_details):
    case = get_case_for_pron(token)
    number = get_number(token)

    for gender in ["masculin", "feminin"]:  
        found_dex_pos = f"Pronume, {gender}, {case}, {number}"
        if found_dex_pos == inflection_dex_details:
            return True
    return False

def inflection_filter(token, inflection_possibilities):
    if '85' in inflection_possibilities:
        return '85'

    for infl in inflection_possibilities:
        inflection_dex_details = mapare.find_dexonline_pos_detail(str(infl))
        if token.pos_ in ["VERB", "AUX"]:  
            if build_inflection_for_verb(token, inflection_dex_details) is True:
                inflection_possibilities = [infl]


        elif token.pos_ in ["NOUN", "ADJ"]:

            if build_inflection_for_noun(token, inflection_dex_details) is True:
                inflection_possibilities = [infl]

        elif token.pos_ in ["PRON", "DET"]:
            
            if build_inflection_for_pron(token, inflection_dex_details) is True:
                inflection_possibilities = [infl]

    return inflection_possibilities

def forme_reflexive_verifier(token: Token) -> str:

    """
        This function will map short reflexive forms into long ones
        using data from reflexive_deps from util_data.py
    """
    word_added = token.text
    if token.dep_ in reflexive_deps:
        case_condition = token.morph.get("Case", ["dummy"])[0] in ["Dat", "Acc"]
        variant_condition = token.morph.get("Variant", ["dummy"])[0] == "Short"
        if case_condition and variant_condition:
            word_added = reflexive_short_to_long_form[token.text]

    return word_added

/Users/inttstbrd/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from transformers import AutoTokenizer, AutoModel
import torch
import logging
logging.getLogger('torch').setLevel(logging.ERROR)
import json
import numpy

all_contexts = json.load(open("util/context.json"))

tokenizer = AutoTokenizer.from_pretrained("dumitrescustefan/bert-base-romanian-uncased-v1", do_lower_case=True)
model = AutoModel.from_pretrained("dumitrescustefan/bert-base-romanian-uncased-v1")


def get_embeddings(text):
        tokenized_text = tokenizer(text, return_tensors="pt")

        with torch.no_grad():
            outputs = model(**tokenized_text)

        word_embeddings = outputs.last_hidden_state
        averaged_embedding = torch.mean(word_embeddings, dim=0)

        return averaged_embedding

cos = torch.nn.CosineSimilarity(dim=0)

def calculate_similarity(input, compared_to):
    if type(input) != "<class 'torch.Tensor'>":
        input=torch.tensor(input)
   
    return cos(input, torch.tensor(compared_to))


def transform_with_mean_pooling(numpy_array, target_shape=(384,)):
    pooled_array = numpy.reshape(numpy_array, (-1, target_shape[0]))
    mean_array = numpy.mean(pooled_array, axis=0) 
    return mean_array


def get_similarity_scores_for_syns(actual_context: list, syn_candidate_context: list, mean: bool, reshapeFlag = True):
    if actual_context is None or syn_candidate_context is None:
        return 0
    meansim = []
    
    if reshapeFlag == True:
        if actual_context[0].shape != (384,):
            actual_context = [transform_with_mean_pooling(numpy.array(act_ctx)) for act_ctx in actual_context]

    for act_ctx in actual_context:
        for syn_ctx in syn_candidate_context:
            actual_similarity = calculate_similarity(input=act_ctx, compared_to=syn_ctx)
            meansim.append(actual_similarity)

    if len(meansim):
        if mean is True:
            return sum(meansim)/len(meansim)
        else:
            return max(meansim)
    else:
        return 0


def get_context_for_each_candidate_syn(token_text, pos_wanted):
    pos_wanted = ud_to_dex[pos_wanted]
    inflected_forms = all_inflected_forms.find_all_inflected_forms(token_text)
    possible_lexeme_ids = find_lexeme_ids(inflected_forms)
    lexeme_ids = find_matching_lexemeIds(possible_lexeme_ids, pos_wanted)
    entry_ids = find_entryIds(lexeme_ids)
    tree_ids = find_treeIds(entry_ids)

    contexts_found = {}

    for treeId in tree_ids:
        contexts_found[treeId] = incarcare_eficienta(treeId)
    return contexts_found   

""" 
    Short demo to show how it actually works. Uncomment and run the main() function.
"""



' \n    Short demo to show how it actually works. Uncomment and run the main() function.\n'

In [5]:
def raw_word(text):
    diacritics = {
      "ă": "a",
      "â": "a",
      "î": "i",
      "ș": "s",
      "ț": "t",
      "č": "c",
      "ş": "s",
      "ž": "z",
      "Ä": "A",
      "Â": "A",
      "Î": "I",
      "Ș": "S",
      "Ț": "T",
      "Č": "C",
      "Ș": "S",
      "Ž": "Z",
      "á": "a",
      "é": "e",
      "í": "i",
      "ó": "o",
      "ú": "u",
      "ű": "u",
      "Á": "A",
      "É": "E",
      "Í": "I",
      "Ó": "O",
      "Ú": "U",
      "Ű": "U",
      "ö": "o",
      "Ö": "O",
      "ü": "u",
      "Ü": "U",
    }

    for k, v in diacritics.items():
        text = text.replace(k, v)
    return text.lower()

def count_consecutive_vowels(word):
    vowels = "aeiouAEIOU"
    consecutive_vowel_count = 0
    total_consecutive_vowels = 0
    for char in word:
        if char in vowels:
            consecutive_vowel_count += 1
        else:
            total_consecutive_vowels += consecutive_vowel_count
            consecutive_vowel_count = 0
        
    total_consecutive_vowels += consecutive_vowel_count
    
    return total_consecutive_vowels


def aproximate_syllables(word: str):
    vowels = "aeiouăîâe"
    groups = ["ch", "gh"]
    word = raw_word(word).lower()
    for group in groups:
        if group == "ch":
            word = word.replace(group, "C")
        elif group == "gh":
            word = word.replace(group, "G")
    
    i = 1
    syllables = []
    last_syllable_index = 0
    while i < len(word) - 1:
        current_char = word[i]
        last_char = word[i-1]
        next_char = word[i+1]
        if i+2 < len(word):
            next2_char = word[i+2]
        # RULE1
        if current_char not in vowels and last_char in vowels and next_char in vowels:
            syllables.append(word[last_syllable_index : i])
            last_syllable_index = i
        # RULE2
        elif current_char not in vowels and next_char not in vowels and last_char in vowels and next2_char in vowels:
            # case 1
            if current_char in "bcdfghptv" and next_char in "lr":
                syllables.append(word[last_syllable_index : i])
            # case 2
            else:
                syllables.append(word[last_syllable_index : i+1])
                i+=1
            last_syllable_index = i
        # RULE3
        elif current_char not in vowels and last_char in vowels:
            cons_group = [current_char]
            j = i + 1 
            while j < len(word):
                if word[j] not in vowels:
                    cons_group.append(word[j])
                else:
                    break
                j+=1
            special_cons_groups = [["l", "p", "t"], ["m", "p", "t"],  ["n", "c", "t"],  ["n", "c", "s"], ["n", "d", "v"], ["r", "c", "t"], ["r", "t", "f"], ["s", "t", "m"]]
            
            # case1
            if cons_group in special_cons_groups:
                syllables.append(word[last_syllable_index:j-1])
                last_syllable_index = j-1
            # case2
            else:    
                syllables.append(word[last_syllable_index:i+1])
                last_syllable_index = i+1
            i=j

        i+=1

    syllables.append(word[last_syllable_index:])
    
    # handle hiat, diftong, triftong
    for syllable in syllables:
        vowels_num = count_consecutive_vowels(syllable)
        if 1 < vowels_num <= 3:
            double_vowel = True
            for i in range(len(syllable) - 1):
                
                if syllable[i] in vowels and syllable[i+1] != syllable[i]:
                    # print(syllable[i])
                    double_vowel = False
            if syllable == syllables[-1] and syllable[-1] == "i" and syllable[i-2] == "i":
                double_vowel = False

            if double_vowel is True:
                syllables.append("dbl_vowel")
        
        elif vowels_num > 3:   
            syllables.append("4vowel")
        # iae, ieu, oeu, oau, eoau, eoeu
        
        for long_hiat in ["iae", "ieu", "oeu", "oau", "eoau", "eoeu"]:
            if long_hiat in syllable:
                syllables.append("lng_hi")

            # print(syllable)
    return syllables




In [7]:
import spacy
nlp = spacy.load("ro_core_news_lg")


In [38]:
from wordfreq import zipf_frequency
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

def choose_meaning(contexts_found, actual_context):
    max_score = -100
    key_to_return = ""
    actual_context = torch.mean(get_embeddings(actual_context), dim=0)

    for key in contexts_found:
        score = get_similarity_scores_for_syns(actual_context=[actual_context], syn_candidate_context=contexts_found[key], mean=False)
        print(score)
        if score > max_score:
            max_score = score
            key_to_return = key

    return key_to_return


def heuristic_comparator(word: str, actual_context: str, token_pos: str, dexonline_order: int, syns_number: int):
    # these can be modified, still testing
    len_weight = -50
    number_of_syllabes_weight = -120
    freq_in_ro_lang_weight = 130
    similarity_with_actual_context_weight = 1500
    dexonline_order_weight = 90
    vowels_num_weight = 60

    def transform_with_mean_pooling(numpy_array, target_shape=(384,)):
        pooled_array = numpy.reshape(numpy_array, (-1, target_shape[0]))
        mean_array = numpy.mean(pooled_array, axis=0) 
        return mean_array

    emb1 = [torch.tensor(transform_with_mean_pooling(numpy.array(get_embeddings(actual_context))))]
   
    sin_context = get_context_for_each_candidate_syn(token_text=word, pos_wanted=token_pos)
    
    similarity_score = 0
    if len(sin_context):
        for key in sin_context:
            similarity_score = get_similarity_scores_for_syns(actual_context=emb1, syn_candidate_context=sin_context[key], mean=False)
    
    try:
        base_form = get_all_forms(nlp(word)[0])[0].get("form")
    except:
        base_form = word

    word_len = len_weight * len(word)
    apx_syllables_number = number_of_syllabes_weight * len(aproximate_syllables(word))
    freq = freq_in_ro_lang_weight * (zipf_frequency(base_form, 'ro')) 
    dexonline_order = dexonline_order_weight * (syns_number - dexonline_order)
    similarity = similarity_with_actual_context_weight * similarity_score
    vowels = vowels_num_weight * sum(1 for letter in word if letter.lower() in 'aeiou')
    
    return word_len + apx_syllables_number + freq + similarity + dexonline_order + vowels


def get_matching_syns(token, actual_context, pos_found):

    try:
        tree_ids, inflection_possibilites, meaning_ids = synonyms_builder_step1(token, pos_found)
    except:
        return []

    contexts_found = {}
        
    for treeId in tree_ids:
        contexts_found[treeId] = incarcare_eficienta(treeId)    
    # print(tree_ids)
    try:
        if len(contexts_found.keys()) > 1:
            chosen_context = str(choose_meaning(contexts_found=contexts_found, actual_context=actual_context))
        else:
            chosen_context = str(list(contexts_found.keys())[0])
        # print(chosen_context)
        sinonime_posibile_dex = get_synonyms(token, [chosen_context])
        
        sinonime_to_return = []
        # for x in sinonime_posibile_dex[:10]:
            # print(x)

        for i in range(len(sinonime_posibile_dex)):
            syn = sinonime_posibile_dex[i]
            if syn == token.text[1:] or syn == sinonime_posibile_dex[i-1][1:]:
                continue
            else:
                sinonime_to_return.append((syn, heuristic_comparator(syn, actual_context, token.pos_, i, len(sinonime_posibile_dex))))
        
        return sorted(sinonime_to_return, key=lambda x: x[1], reverse=True)
    
    except IndexError:
        print("no synonyms")
        pass


def get_syns(target):
    doc = nlp(target.doc)
    print(target, "!!!")
    for token in doc:
        if token == target:
            pos_found = ud_to_dex[token.pos_]
            syns = get_matching_syns(token, actual_context=doc.text, pos_found=pos_found)
            try:
                return syns[:10]
            except:
                return []
    return []

Token.set_extension("get_syns", method=get_syns, force=True)

# force meaning id ca filter pentru verb ca uram (aduci sinonimele in functie de chosen_context si fortezi)


In [15]:

from spacy.tokens import Doc
def oltenizare_worker(doc: Doc) -> list[str]:
    """
    This function will find every at a perfect present tense and
    turn it into its past perfect form.
    """
    new_phrase = []
    actual_pers = ""
    actual_num = ""
    # print("OLTENIZARE")
    docgen = iter(range(0, len(doc)))
    for i in docgen:
        if doc[i].pos_ not in banned_pos:
            if doc[i].dep_ == "nsubj" or doc[i].dep_ == "nsubj:pass":
                # extract data from the phrase-subject to get right form
                # of verb there will be needed for person and number
                actual_num, actual_pers = get_right_person_and_number(doc[i])
                new_phrase.append(doc[i].text if doc[i].text != "s" else "se")

            elif (
                doc[i].dep_ == "ROOT"
                and doc[i-1].dep_ in {"aux", "aux:pass", "cop"} 
                and doc[i-2].dep_ == "aux"
            ):
                # handle cases like these: "Eu am fost plecat."

                if actual_pers == "" and actual_num == "":
                    actual_num, actual_pers = get_right_person_and_number(doc[i-2])

                if doc[i].pos_ != "VERB":
                    new_phrase.append(doc[i].text)
               
                elif doc[i-1].morph.get("VerbForm") == ["Part"]:
                    new_phrase.append(
                        get_wanted_form(
                            doc[i-1], "perfect simplu", actual_pers, actual_num
                        )
                    )
                    new_phrase.append(doc[i].text)
                elif doc[i-1].morph.get("VerbForm") == ["Inf"]:
                    new_phrase += [doc[i-2].text, doc[i-1].text, doc[i].text]
                    i += 2
                
                elif doc[i].morph.get("VerbForm") == ["Inf"]:
                    new_phrase += [doc[i].text]

            elif (
                doc[i].dep_ == "ROOT"
                and doc[i-1].dep_ == "cc"
                and doc[i-2].dep_ == "aux"
            ):
                
                if actual_pers == "" and actual_num == "":
                    actual_num, actual_pers = get_right_person_and_number(doc[i-2])
                new_phrase.append(
                    get_wanted_form(
                                    doc[i],
                                    "perfect simplu",
                                    actual_pers,
                                    actual_num
                                )
                )
            
            elif (
                doc[i].dep_ == "ROOT"
                and doc[i-2].dep_ == "aux"
            ):
                
                if actual_pers == "" and actual_num == "":
                    actual_num, actual_pers = get_right_person_and_number(doc[i-2])
                
                if doc[i].morph.get("VerbForm") == ["Part"]:
                    new_phrase.append(
                        get_wanted_form(
                                        doc[i],
                                        "perfect simplu",
                                        actual_pers,
                                        actual_num
                                    )
                    )
                else:
                    new_phrase.append(doc[i].text)

            elif doc[i].dep_ in root_forms and doc[i-1].dep_ == "aux":
                if doc[i-2].dep_ == "aux":
                    if len(new_phrase) > 0 and doc[i-2].text == new_phrase[-1]:            
                        new_phrase += [doc[i-1].text, doc[i].text]
                    else:
                        new_phrase += [doc[i-2].text, doc[i-1].text, doc[i].text]
            
                    i += 2

                else:
                    # handle cases like these: "Eu am plecat"
                    # ensure that the construction found
                    # (aux + verb) is not at a future tense
                
                    if doc[i].morph.get("VerbForm", ["UNKNONWN"])[0] in {"Part"}:
                        if doc[i-1].pos_ == "AUX":
                            # if actual_pers == "" and actual_num == "":
                                # if person and number paramateres cant be
                                # found from subject of a phrase,
                                # the verb will get this from its inflection
                            (
                                actual_num,
                                actual_pers,
                            ) = get_right_person_and_number(doc[i-1])
                            # print(get_right_person_and_number(doc[i-1]))
                            word_to_add = get_wanted_form(
                                    doc[i],
                                    "perfect simplu",
                                    actual_pers,
                                    actual_num,
                                )
                            if word_to_add == "UNKNOWN":
                                if doc[i-1].lemma_ == "avea":
                                    new_phrase.append(doc[i-1].text)
                                    
                                new_phrase.append(
                                    doc[i].text
                                )
                            else:
                                new_phrase.append(word_to_add)

                        else:
                            # trick to handle exceptions found
                            if actual_pers == "" and actual_num == "":
                                (
                                    actual_num,
                                    actual_pers,
                                ) = get_right_person_and_number(doc[i-1])
                            new_phrase.append(doc[i].text)

                    else:
                        if doc[i].dep_ != "cop":
                            if len(new_phrase) > 0:
                                if doc[i-1].text != new_phrase[-1]:
                            # the construction is at a future tense or cond opt
                                    new_phrase.append(doc[i-1].text)
                            else:
                                new_phrase.append(doc[i].text)
                            
                        new_phrase.append(doc[i].text)
                        

            elif doc[i].dep_ == "aux:pass" and doc[i].lemma_ == "fi":
                    # AM COMENTAT PT EXEMPLUL ASTA El va pleca, dar ieri ar fi stat apoi a plecat.
                if doc[i-1].dep_ == "aux":
                    if doc[i+1].dep_ in root_forms and doc[i+1].pos_ not in {"VERB", "AUX"}:
                        (
                            actual_num,
                            actual_pers,
                        ) = get_right_person_and_number(doc[i-1])
                        new_phrase.append(get_wanted_form(
                                    doc[i],
                                    "perfect simplu",
                                    actual_pers,
                                    actual_num,
                                ))
                    elif doc[i+1].dep_ not in {"ROOT"}:
                    # else:
                        new_phrase.append(doc[i].text)
                    pass

                else:
                    new_phrase.append(doc[i].text)

            elif doc[i].dep_ == "det" and doc[i].pos_ == "AUX" and doc[i+1].dep_ == "dep" and doc[i+1].pos_ == "AUX":
                (
                    actual_num,
                    actual_pers,
                ) = get_right_person_and_number(doc[i])

                new_phrase.append(
                    get_wanted_form(
                                    doc[i+1],
                                    "perfect simplu",
                                    actual_pers,
                                    actual_num,
                                )
                )
                next(docgen)


            elif doc[i].dep_ == "aux":
                add = True
                anc_verb = {}
                for t in doc[i].ancestors:
                    if t.pos_ in {"VERB", "AUX"}:
                        add = False
                        anc_verb = t
                        
                if anc_verb == {}:
                    anc_verb=t

                if doc[i+1].dep_ == "mark" and doc[i+1].pos_ in {"ADP", "PART"} and doc[i+2].dep_ == "ROOT" and doc[i+2].pos_ == "VERB":
                    new_phrase += [doc[i].text, doc[i+1].text, doc[i+2].text]
                    next(docgen)
                    next(docgen)
                    # next(docgen)
                    # continue

                elif doc[i+1].morph.get("VerbForm") == ["Part"] or anc_verb.morph.get("VerbForm") == ["Part"]:
                    if doc[i].morph.get("VerbForm") in [["Fin"], ["Inf"]]:
                        new_phrase.append(doc[i].text)
                        new_phrase.append(doc[i+1].text)
                        next(docgen)
                    elif doc[i].text in {"ar", "ai", "am", "aș", "ați"} and doc[i+1].morph.get("VerbForm") in [["Fin"], ["Inf"]]:
                        new_phrase.append(doc[i].text)
                        new_phrase.append(doc[i+1].text)
                        new_phrase.append(doc[i+2].text)
                        next(docgen)
                        next(docgen)
                   
                    
                elif add is True or doc[i+1].morph.get("VerbForm") == ["Inf"] or anc_verb.morph.get("VerbForm") == ["Inf"]:
                    # print(new_phrase[-1], doc[i].text)
                    if len(new_phrase) > 0:
                        if new_phrase[-1] != doc[i].text:
                            new_phrase.append(doc[i].text)
                    else:
                        new_phrase.append(doc[i].text)
                else:
                    new_phrase.append(doc[i].text)
               
            # elif doc[i].dep_ == "dep" and doc[i].dep_ == "dep" and doc[i].pos_ == and doc[i].dep
                        # A FOST A MAMEI
                
            else:
                if doc[i].pos_ == "PRON" or doc[i].dep_ == "expl":
                    if doc[i].text[-1] == "-":

                        try:
                            anc = list(doc[i].ancestors)[0]                            
                            if anc.morph.get("VerbForm") == ["Part"]:
                                word_added = forme_reflexive_verifier(doc[i])
                            else:
                                word_added = doc[i].text
                        except:
                            word_added = doc[i].text
                    else:
                        word_added = doc[i].text
                else:
                    word_added = doc[i].text

                # print(word_added, doc[i].pos_, doc[i].dep_,)
                new_phrase.append(word_added)

        else:
            new_phrase.append(doc[i].text)
            actual_num, actual_pers = "", ""

    return new_phrase


def oltenizare(doc: Doc) -> str:
    """
        This function builds a new phrase with the verbs modified by the oltenizare_worker flow.
    """
    new_phrase = oltenizare_worker(doc)
    phrase_to_return = ""

    for i in range(len(new_phrase)):
        new_phrase[i] = new_phrase[i].replace("n-", "nu")
        new_phrase[i] = new_phrase[i].replace("l-", "îl")
        # new_phrase[i] = new_phrase[i].replace("s-", "se")
        new_phrase[i] = new_phrase[i].replace("ţi-", "îţi")
        # new_phrase[i] = new_phrase[i].replace("i-", "îi")
        new_phrase[i] = new_phrase[i].replace("le-", "le")
        new_phrase[i] = new_phrase[i].replace("c-", "că")
        
        # building the initial phrase back following the next
        # rule: word, word (or any other PUNCT)
        # edge-case 1: for "-" where the rule is word-word
        # edge-case 2: word. Word (the same for ?, !, \n)
        # print(new_phrase[i])
        if "-" in new_phrase[i][1:]:
            phrase_to_return += " " + new_phrase[i]
        elif "-" == new_phrase[i][0]:
            phrase_to_return += "-"+new_phrase[i]

        elif not new_phrase[i].isalpha():
            phrase_to_return += new_phrase[i]

        else:
            if new_phrase[i-1] in end_of_phrase:
                phrase_to_return += " " + new_phrase[i].capitalize()
            elif new_phrase[i-1][-1] == "-":
                phrase_to_return += new_phrase[i]
            else:
                phrase_to_return += " " + new_phrase[i]

    phrase_to_return = phrase_to_return.replace("--", "-")
    
    words = phrase_to_return.split()
    phrase = []
    for word in words:
        if word.endswith("-o"):
            word_modfied = f"o {word[:-2]}"
        else:
            word_modfied = word
        phrase.append(word_modfied)
    return " ".join(phrase)

Doc.set_extension("oltenizare", method=oltenizare, force=True)


In [12]:


import time
import spacy
t1 = time.time()

# pentru teste
cuv = "deplasam"
cuv2="uram"
# contextele pe care vreau sa l testez
actual_context = "Noi ne-am deplasat la pas."

doc = nlp(actual_context)

def ltrdict(phrase):
    d = {}
    for chr in phrase:
        if chr.isalpha():
            if chr not in d:
                d[chr] = 0
            else:
                d[chr] += 1
    return d


def oltenizare_exp(file):
    oltenizate = 0
    f = open(file, "r")
    props = f.readlines()
    for prop in props:
        prop = prop.lower()
        oltenizat_prop = nlp(prop)
        
        oltenizat_prop = oltenizare(oltenizat_prop)
        if ltrdict(prop) != ltrdict(oltenizat_prop.lower()):
            print(oltenizat_prop.lower(), prop)
    
    
# oltenizare_exp("avioane.txt")
# doc = nlp("sunt surprinsă că te-ai întors după mine.")
# doc = nlp("m-aș fi pus pe plâns, ca o stropitoare? ")

# for token in doc:
#     print(token, token.morph, token.lemma_, token.dep_, token.pos_)

# print()
# print()
    
# print(oltenizare(doc))

# Future work
# in ramura else adaugi forme reflexive doar daca sunt pronume, muta replace-urile din newphrase in oltenizator worker

# vazut-o =>>>> o vazui

In [39]:
# EDGE CASES CAUSED BY SPACY 
# de ce am fugi pe acolo?


ex = [
    # "Eu aș fi mers la mare.",
    "Noi am plecat la mare.",
    "Eu mergeam la mare.", 
    "Era bine să fi plecat la mare.",
    "Ei au vizitat marea.",
    "Eu voi fi vizitat marea.",
    "Eu voi merge la mare.",
    "El s-a dus la mare.",
    "Eu mi-am vizitat prietenii.",
]

doc = nlp(ex[0])
index=2
print(doc, doc[index])
syn_list = doc[index]._.get_syns()
for syn in syn_list:
    print(syn)

# for prop in ex:
#     prop = nlp(prop)
#     print(prop, prop._.oltenizare())

Noi am plecat la mare. plecat
plecat !!!
[('plecat', tensor(5147.1482, dtype=torch.float64)), ('pornit', tensor(4975.2128, dtype=torch.float64)), ('ieșit', tensor(4902.3925, dtype=torch.float64)), ('dispărut', tensor(4646.7599, dtype=torch.float64)), ('evadat', tensor(4602.8228, dtype=torch.float64)), ('risipit', tensor(4435.5997, dtype=torch.float64)), ('răspândit', tensor(4180.5773, dtype=torch.float64)), ('aplecat', tensor(4010.2930, dtype=torch.float64)), ('îndoit', tensor(3986.0682, dtype=torch.float64)), ('răzleț', 3750.0)]


In [ ]:

propozitii = [
    "m-aș fi pus pe plâns, ca o stropitoare?",
    "sunt surprinsă că te-ai întors după mine.",
    "dacă tu socoteşti că a fi încătuşat, jefuit,",
    "nu-şi mai făcea griji că va fi ucis.",
    "succesul tău a fost puţin şi succesul meu.",
    "Eu am plecat",
    "El va fi venit",
    "El ar fi venit, dar nu",
    "nu l-am mai văzut pe aici.",
    "cenușăreasa va fi surprinsă.",
    "nu a mai rămas nimic în ce să crezi.",
    "atunci, prin ordin regal, acea fată va fi soția prințului.",
    "dacă i-ai dat o șansă inimii tale",
    "El va pleca, dar ieri ar fi stat apoi a plecat.",
    "vor deveni realitate.",
    "îți dai seama ce ai spus",
    "o voi citi eu!",
    "dar a fost așa de frumos!",
    "vom mai vedea",
    "a fost a mamei",
    "ar fi bine să te descotorosești de aceste vise.",
    "Nu mă îndoiesc că ați avut în minte întreaga scenă.",
    "cred că am uitat de toate!",
    "tot va fi interesat de una din ele, nu-i așa?", 
    "am să stau tot după tine.",
    "nu-ţi fă griji pentru mine, e ziua ta cea mare.",
    "tu ai copite.",
    "eu am palme."
]

In [175]:
f = open("multilex_test_it_combined_labels.tsv", "r")
rows = f.readlines()
phrases = []

for row in rows:
    row = row.split("\t")
    phrases.append({
        "id": row[0],
        "language": row[1],
        "phrase": row[2],
        "target": row[3],
        "score": row[4],
        "alternatives": row[5:],
    })

print(phrases[0])

FileNotFoundError: [Errno 2] No such file or directory: 'multilex_test_it_combined_labels.tsv'

In [176]:
import deepl

def translate_to_ro(phrase, source_lang="EN"):
    auth_key = "663d15b4-e26d-49b5-8d54-eae73b5d5f03:fx"  
    translator = deepl.Translator(auth_key)

    result = translator.translate_text(phrase, source_lang=source_lang, target_lang="RO")
    return result.text  


In [177]:
phrases_in_ro = []
id_act = 31
from tqdm import tqdm

for phrase in tqdm(phrases):
    alternatives_in_en = phrase.get("alternatives")
    alternatives_in_ro = [translate_to_ro(alternative) for alternative in alternatives_in_en]

    phrases_in_ro.append({
        "id": f"ro_{id_act}",
        "language": "romanian",
        "phrase": translate_to_ro(phrase.get("phrase")),
        "target": translate_to_ro(phrase.get("target")),
        "score": translate_to_ro(phrase.get("score")),
        "alternatives": alternatives_in_ro
    })

    id_act += 1

# print(phrases_in_ro)

  0%|          | 0/144 [00:00<?, ?it/s]


AttributeError: 'str' object has no attribute 'get'

In [178]:
g = open("ro_data_MLSP.txt", "w")

for phrase in phrases_in_ro:
    id = phrase.get("id")
    language = phrase.get("language")
    sentence = phrase.get("phrase")
    target = phrase.get("target")
    score = phrase.get("score")
    alternatives = phrase.get("alternatives")
    string_to_append = f"{id}\t{language}\t{sentence}\t{target}\t{score}\t"
    for alt in alternatives:
        string_to_append += alt + "\t"
    print(string_to_append)
    g.write(string_to_append)


In [179]:
phrases_with_targets = []
g = open("ro_data_MLSP_target_in_phrase.txt", "w")


def get_syns(phrase, target):
    doc = nlp(phrase)
    for token in doc:
        if token.text == target:
            pos_found = ud_to_dex[token.pos_]
            syns = get_matching_syns(token, actual_context=doc.text, pos_found=pos_found)
            try:
                return syns[:10]
            except:
                return []
    return []

# for phrase in tqdm(phrases_in_ro):
#     id = phrase.get("id")
#     language = phrase.get("language")
#     sentence = phrase.get("phrase")
#     target = phrase.get("target")
#     score = phrase.get("score")
#     alternatives = phrase.get("alternatives")
    
#     string_to_append = f"{sentence}\t{target}"
#     for alt in alternatives:
#         string_to_append += "\t" + alt
    
#     syns = get_syns(sentence, target)
#     if syns is not None:
#         if target in sentence and len(syns) > 0:
#             phrases_with_targets.append(phrase)
#             g.write(string_to_append)
    


In [180]:
f = open("multilex_test_all_combined_labels.tsv", "r")
scrie_in_it_gold_Data = open("ro_data_MLSP_target_in_phrase_ITALIAN.txt", "w")

for row in f.readlines():
    row = row.split("\t")
    if "it" in row[0]:
        string_to_append = f"{row[2]}\t{row[3]}"
        for alt in row[5:]:
            string_to_append += "\t" + alt
        print(string_to_append)
        scrie_in_it_gold_Data.write(string_to_append)

FileNotFoundError: [Errno 2] No such file or directory: 'multilex_test_all_combined_labels.tsv'

In [31]:
# scrie traducerile de la shared task italiana in ro
f = open("ro_data_MLSP_target_in_phrase_ITALIAN.txt", "r")

data = f.readlines()
data_in_ro_from_it = []

for row in tqdm(data):
    row = row.split("\t")
    phrase = translate_to_ro(row[0], source_lang="IT")
    target = translate_to_ro(row[1], source_lang="IT")

    alternatives_in_ro = []
    alternatives = row[2:]
    for alt in alternatives:
        alternatives_in_ro.append(translate_to_ro(alt, source_lang="IT"))

    data_in_ro_from_it.append((phrase, target, alternatives_in_ro))

print(data_in_ro_from_it)

  0%|          | 0/561 [00:00<?, ?it/s] [INFO] 2024-05-28 21:02:16,675 deepl:49 - Request to DeepL API method=POST url=https://api-free.deepl.com/v2/translate
 [INFO] 2024-05-28 21:02:17,324 deepl:49 - DeepL API response status_code=200 url=https://api-free.deepl.com/v2/translate
 [INFO] 2024-05-28 21:02:17,328 deepl:49 - Request to DeepL API method=POST url=https://api-free.deepl.com/v2/translate
 [INFO] 2024-05-28 21:02:17,733 deepl:49 - DeepL API response status_code=200 url=https://api-free.deepl.com/v2/translate
 [INFO] 2024-05-28 21:02:17,736 deepl:49 - Request to DeepL API method=POST url=https://api-free.deepl.com/v2/translate
 [INFO] 2024-05-28 21:02:18,132 deepl:49 - DeepL API response status_code=200 url=https://api-free.deepl.com/v2/translate
 [INFO] 2024-05-28 21:02:18,136 deepl:49 - Request to DeepL API method=POST url=https://api-free.deepl.com/v2/translate
 [INFO] 2024-05-28 21:02:18,535 deepl:49 - DeepL API response status_code=200 url=https://api-free.deepl.com/v2/tra

[('Armele și dragostea sunt inseparabile și răspund viziunii hedoniste și laice a vieții tipice Renașterii.', 'inseparabil', ['inseparabil', 'inseparabil', 'inseparabil', 'inseparabil', 'inseparabil', 'inseparabil', 'inseparabil', 'indivizibil', 'indivizibil', 'indivizibil', 'indivizibil', 'indivizibil', 'indivizibil', 'indisolubilă\n']), ('Armele și dragostea sunt inseparabile și răspund viziunii hedoniste și laice a vieții tipice Renașterii.', 'viziune', ['idee', 'idee', 'idee', 'imagine', 'imagine', 'imagine', 'aspect', 'concept', 'percepție', 'perspectivă', 'uite\n']), ('Armele și dragostea sunt inseparabile și răspund viziunii hedoniste și laice a vieții tipice Renașterii.', 'laic', ['seculară', 'seculară', 'areligioasă', 'ateu\n']), ('Dar la aceste întrebări ar fi putut răspunde și un țipăt furibund, inuman: atât de puternic încât întregul oraș, oricât de mult ar mai fi rămas în picioare dincolo de fundalul intact și înșelător al Via Mazzini până la îndepărtatele Ziduri sfărâmate

In [181]:
# golddata pt romana doar cu frazele care au si sinonime in romana

g = open("GOLD_DATA_RO_FROM_IT.txt", "w")

for phrase in tqdm(data_in_ro_from_it):
    sentence = phrase[0]
    target = phrase[1]
    string_to_append = f"{phrase[0]}\t{phrase[1]}"
    alternatives = phrase[2]

    for alt in alternatives:
        string_to_append += "\t" + alt
    
    syns = get_syns(sentence, target)
    if syns is not None:
        if target in sentence and len(syns) > 0:
            phrases_with_targets.append(phrase)
            g.write(string_to_append)

NameError: name 'data_in_ro_from_it' is not defined

In [183]:
# # pentru frazele din ro_data_mlsp aplica LS iar si evalueaza

# rezultate actuale:
# =========   EVALUATION config.=========
# GOLD file = ro_data_MLSP_target_in_phrase.txt
# PREDICTION LABELS file = ls_ro.txt
# OUTPUT file = aici.txt
# ===============   RESULTS  =============

# MAP@1/Potential@1/Precision@1 = 0.3333

# MAP@3 = 0.1827
# MAP@5 = 0.1252
# MAP@10 = 0.0646

# Potential@3 = 0.4777
# Potential@5 = 0.5
# Potential@10 = 0.5222

# Accuracy@1@top_gold_1 = 0.2666
# Accuracy@2@top_gold_1 = 0.3888
# Accuracy@3@top_gold_1 = 0.4222

In [182]:
#GENEREAZA LS PT RO FROM EN

f = open("GOLD_DATA_RO_FROM_EN.txt", "r")
g = open("ls_ro.txt", "w")
from tqdm import tqdm

for row in tqdm(f.readlines()):
    row = row.split("\t")
    romanian_sentence = row[0]
    target = row[1]
    string_to_write = f"{romanian_sentence}\t{target}\t"
    syns = get_syns(romanian_sentence, target)

    for syn in syns:
        string_to_write += syn[0] + "\t"
    print(string_to_write)
    g.write(string_to_write + "\n")

    


FileNotFoundError: [Errno 2] No such file or directory: 'GOLD_DATA_RO_FROM_EN.txt'

In [44]:
#GENEREAZA LS PT RO FROM EN

f = open("GOLD_DATA_RO_FROM_IT.txt", "r")
scrie_it = open("ls_ro_it.txt", "w")
from tqdm import tqdm
i=0
for row in tqdm(phrases_with_targets):
    # row = row.split("\t")
    romanian_sentence = row[0]
    target = row[1]
    string_to_write = f"{romanian_sentence}\t{target}\t"
    syns = get_syns(romanian_sentence, target)

    for syn in syns:
        string_to_write += syn[0] + "\t"

    print(string_to_write)
    scrie_it.write(string_to_write + f"\n")

    


  1%|          | 1/91 [00:00<00:31,  2.89it/s]

...astfel s-a răspândit printre scapigliati un sentiment de rebeliune și de dispreț radical față de standardele și convingerile morale curente, ceea ce a avut drept consecință crearea mitului vieții dizolvate și neregulate (așa-numitul maledettismo).	dispreț	necaz	dezgust	sentiment	


  2%|▏         | 2/91 [00:02<01:59,  1.34s/it]

La Scapigliati se formează un fel de conștiință dualistă (un vers al lui Arrigo Boito se intitulează tocmai Dualism) care subliniază contrastul strident dintre "idealul" pe care se dorește a-l atinge și "adevăratul", realitatea crudă, descrisă în mod obiectiv și anatomic.	strident	subțire	viu	aspru	nepotrivit	nearmonios	distonant	puternic	ascuțit	țipător	intens	


  3%|▎         | 3/91 [00:02<01:13,  1.19it/s]

...

...Vocea izolată a lui Nietzsche a prezis declinul Occidentului și nașterea unei noi umanități, inițiind o direcție nihilistă care va fi reprodusă în secolul XX.	prezis	ghicit	prezis	prevestit	profetizat	


 98%|█████████▊| 89/91 [01:28<00:00,  2.16it/s]

Potrivit romanticilor, infinitul generează în om un sentiment de teroare și neputință, definit ca sublim...	teroare	groază	oroare	înfiorare	frică	spaimă	încrâncenare	îngrozire	înfricoșare	înspăimântare	
La apogeul umanismului, problema limbii a devenit mai aprinsă, nu în ultimul rând din cauza apariției tiparului, care a necesitat un standard național coerent și omogen.	standard	nivel	stadiu	


100%|██████████| 91/91 [01:28<00:00,  1.03it/s]

Cu toate acestea, această clasificare nu ar trebui înțeleasă într-un sens strict cronologic, ci mai degrabă ar trebui privită din punct de vedere conceptual.	strict	aspru	fix	sever	exact	riguros	științific	
Cu toate acestea, această clasificare nu ar trebui înțeleasă într-un sens strict cronologic, ci mai degrabă ar trebui privită din punct de vedere conceptual.	conceptual	abstract	noțional	speculativ	


In [184]:
# genereaza ls pt all using dex

#GENEREAZA LS PT RO FROM EN

f = open("GOLD_DATA_ALL.txt", "r")
scrie_all = open("ls_ro_all.txt", "w")
from tqdm import tqdm
i=0
all_phrases = [x.split("\t") for x in f.readlines()]

for row in tqdm(all_phrases):
    # row = row.split("\t")
    romanian_sentence = row[0]
    target = row[1]
    string_to_write = f"{romanian_sentence}\t{target}\t"
    syns = get_syns(romanian_sentence, target)

    for syn in syns:
        string_to_write += syn[0] + "\t"

    print(string_to_write)
    scrie_all.write(string_to_write + f"\n")

    


FileNotFoundError: [Errno 2] No such file or directory: 'GOLD_DATA_ALL.txt'

In [185]:
f = open("GOLD_DATA_ALL.txt", "r")
scrie_all = open("ls_ro_all_v2.txt", "w")
from tqdm import tqdm
i=0
all_phrases = [x.split("\t") for x in f.readlines()]

for row in tqdm(all_phrases):
    # row = row.split("\t")
    romanian_sentence = row[0]
    target = row[1]
    string_to_write = f"{romanian_sentence}\t{target}\t"
    syns = get_syns(romanian_sentence, target)

    for syn in syns:
        string_to_write += syn[0] + "\t"

    print(string_to_write)
    scrie_all.write(string_to_write + f"\n")


FileNotFoundError: [Errno 2] No such file or directory: 'GOLD_DATA_ALL.txt'

In [186]:
# bring alternatives to same inflexion using dexonline framework

import spacy
nlp = spacy.load("ro_core_news_sm")

def parse_phrase(phrase):
    phrase = phrase.split("\t")
    actual_phrase = phrase[0]
    target = phrase[1]
    alternatives = phrase[2:]

    return nlp(actual_phrase), target, alternatives

def get_inflection_details(inflectionId):
    infl_aux = mapare.find_dexonline_pos_detail(str(inflectionId)).split(", ")[1:]
    target_inflection = ""
        
    if len(infl_aux) > 1:
        for att in infl_aux[:-1]:
            target_inflection +=  att + ", "
        target_inflection += infl_aux[-1]
    else:
        target_inflection = ""

    return target_inflection

def get_alternatives_at_same_inflection(phrase):    
    parsed, target, alternatives = parse_phrase(phrase)
    
    for t in parsed:
        if t.text == target:
            token=t

    token_text = re.sub('[^a-zA-ZăâîșțĂÂÎȘȚ]', '', token.text.lower())
    inflected_forms = all_inflected_forms.find_all_inflected_forms(token_text)
    pos_wanted = ud_to_dex[token.pos_]
    inflection_possibilities = find_inflection_possibilites(token, inflected_forms, pos_wanted)

    if len(inflection_possibilities) > 1:
        inflection_possibilities =  inflection_filter(token=token, inflection_possibilities=inflection_possibilities)

    alt_inflected = []
    
    for inflectionId in inflection_possibilities:
        target_inflection = get_inflection_details(inflectionId)

        for alt in alternatives:
            aux_phrase = phrase.replace(target, alt)
            aux_doc = nlp(aux_phrase)
            inflected_alt = ""
            alt_token = aux_doc[-1]
            for t in aux_doc:
                alt_inflection = ""
                if t.text == alt:
                    token_text = re.sub('[^a-zA-ZăâîșțĂÂÎȘȚ]', '', t.text.lower())
                    inflected_forms = all_inflected_forms.find_all_inflected_forms(token_text)
                    if inflected_forms == ['unidentified']:
                        continue
                    else:
                        pos_wanted = ud_to_dex[token.pos_]
                        inflection_possibilities = find_inflection_possibilites(t, inflected_forms, pos_wanted)

                        if len(inflection_possibilities) > 1:
                            inflection_possibilities =  inflection_filter(token=t, inflection_possibilities=inflection_possibilities)
                        else:
                            inflection_possibilities = ["noid"]
                        
                        alt_inflection = get_inflection_details(inflection_possibilities[0])
                        alt_token = t
            
            if target_inflection == alt_inflection:
                alt_inflected.append(alt)

            else:
                # print("modifica aici ceva pentru: ", alt_token, "cu alternativele: ", alternatives)
                all_forms_for_alt = get_all_forms(alt_token)

                for el in all_forms_for_alt:
                    # print(el)
                    if target_inflection in el.get('pos'):
                        # print(el)
                        form = el.get('form')
                        alt_inflected.append(form)
                        break
                
        

            # if inflected_alt != "":
            #     alt_inflected.append(inflected_alt)
    
    if alt_inflected:
        return alt_inflected
    else:
        return alternatives
    
f = open("/Users/inttstbrd/Desktop/licenta/nlp_lic/dexonline/ls_experiments/rollm_LS.txt", "r")
phrases = f.readlines()

for phrase in phrases:
    print("Alternatives: ", get_alternatives_at_same_inflection(phrase))




Alternatives:  ['antipatie', 'dispreț']
Alternatives:  ['ascuțit', 'ascuțit', 'ascuțit', 'ascuțit\n']
Alternatives:  ['compactat', 'constrâns', 'concentrat', 'concentrat\n']
Alternatives:  ['impoliticos', 'impoliticos', 'impoliticos', 'impoliticos\n']
Alternatives:  ['incertitudine', 'incertitudine', 'incertitudine']
Alternatives:  ['compozit', 'compozit', 'compozit', 'compozit\n']
Alternatives:  ['rafinat', 'sofisticat', 'sofisticat', 'sofisticat\n']
Alternatives:  ['întuneric', 'întunecat', 'întunecat\n']
Alternatives:  ['modificări', 'schimbări', 'schimbări', 'modificări', 'schimbări', 'schimbări', 'modificări', 'schimbări', 'schimbări']
Alternatives:  ['încurajare', 'încurajare', 'încurajare']
Alternatives:  ['surpriză', 'surpriză', 'surpriză']
Alternatives:  ['circulat', 'circulat', 'circulat', 'circulat\n']
Alternatives:  ['incorectă', 'incorectă', 'incorectă']
Alternatives:  ['ascuns', 'ascuns', 'ascuns', 'ascuns', 'ascuns\n']
Alternatives:  ['idiot', 'ignorant', 'incompetent', 

In [198]:
# f = open("GOLD_DATA_ALL.txt", "r")
scrie_all = open("romanian_lcp_with_dex.txt", "w")
from tqdm import tqdm
i=0
import pandas as pd
import torch
import numpy
import tqdm
# from dexonline.prototype import get_syns
df = pd.read_csv('/Users/inttstbrd/Desktop/licenta/nlp_lic/dexonline/ls_experiments/romanian_lcp.tsv', sep='\t')


alt_list = []

for i in tqdm.tqdm(range(len(df))):
    row = df.iloc[i]
    romanian_sentence = row[1]
    target = row[2]
    try:
        syns = get_syns(romanian_sentence, target)
        alt_list.append(syns)
    except:
        alt_list.append("")

  0%|          | 0/569 [00:00<?, ?it/s]

/var/folders/l9/4jrhpypx0fq40ckdnkdrqvmh0000gn/T/ipykernel_52164/3623798018.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  romanian_sentence = row[1]
/var/folders/l9/4jrhpypx0fq40ckdnkdrqvmh0000gn/T/ipykernel_52164/3623798018.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  target = row[2]
/var/folders/l9/4jrhpypx0fq40ckdnkdrqvmh0000gn/T/ipykernel_52164/3623798018.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  romani

no synonyms


  2%|▏         | 11/569 [00:05<04:21,  2.13it/s]/var/folders/l9/4jrhpypx0fq40ckdnkdrqvmh0000gn/T/ipykernel_52164/3623798018.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  romanian_sentence = row[1]
/var/folders/l9/4jrhpypx0fq40ckdnkdrqvmh0000gn/T/ipykernel_52164/3623798018.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  target = row[2]
  2%|▏         | 12/569 [00:05<04:23,  2.11it/s]/var/folders/l9/4jrhpypx0fq40ckdnkdrqvmh0000gn/T/ipykernel_52164/3623798018.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (

tensor(0.0251, dtype=torch.float64)
tensor(-0.0153, dtype=torch.float64)


...

 98%|█████████▊| 555/569 [03:55<00:03,  4.42it/s]/var/folders/l9/4jrhpypx0fq40ckdnkdrqvmh0000gn/T/ipykernel_52164/3623798018.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  romanian_sentence = row[1]
/var/folders/l9/4jrhpypx0fq40ckdnkdrqvmh0000gn/T/ipykernel_52164/3623798018.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  target = row[2]
/var/folders/l9/4jrhpypx0fq40ckdnkdrqvmh0000gn/T/ipykernel_52164/3623798018.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access 

no synonyms
no synonyms


/var/folders/l9/4jrhpypx0fq40ckdnkdrqvmh0000gn/T/ipykernel_52164/3623798018.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  romanian_sentence = row[1]
/var/folders/l9/4jrhpypx0fq40ckdnkdrqvmh0000gn/T/ipykernel_52164/3623798018.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  target = row[2]
 99%|█████████▊| 561/569 [03:56<00:01,  4.83it/s]/var/folders/l9/4jrhpypx0fq40ckdnkdrqvmh0000gn/T/ipykernel_52164/3623798018.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access 

no synonyms


 99%|█████████▉| 566/569 [03:57<00:00,  5.63it/s]/var/folders/l9/4jrhpypx0fq40ckdnkdrqvmh0000gn/T/ipykernel_52164/3623798018.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  romanian_sentence = row[1]
/var/folders/l9/4jrhpypx0fq40ckdnkdrqvmh0000gn/T/ipykernel_52164/3623798018.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  target = row[2]
/var/folders/l9/4jrhpypx0fq40ckdnkdrqvmh0000gn/T/ipykernel_52164/3623798018.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access 

In [199]:
df['dex_suggestions'] = alt_list

In [200]:
print(df)

         id                                           sentence          word  \
0    en_100  Capitolul al doilea investighează această legă...    gimnastica   
1    en_101  Capitolul al doilea investighează această legă...     amănunțit   
2    en_102  Încurajat de susținătorii din sud (mulți dintr...   răsunătoare   
3    en_103  Încurajat de susținătorii din sud (mulți dintr...        dintre   
4    en_104  Încurajat de susținătorii din sud (mulți dintr...  susținătorii   
..      ...                                                ...           ...   
564   en_95  Până în anii 1960, tot mai multe trimiteri poș...        masini   
565   en_96  Calcularea unor astfel de capacitanțe și explo...    contracara   
566   en_97  Calcularea unor astfel de capacitanțe și explo...     electrice   
567   en_98  Calcularea unor astfel de capacitanțe și explo...   capacitanțe   
568   en_99  Capitolul al doilea investighează această legă...     discutând   

     start  end  complexity  annot0  an

In [201]:
df.to_csv('romanian_lcp_with_dex.tsv', sep='\t', index=False)


In [208]:

import spacy
# from dexonline.prototype import *


def get_syns(phrase, target):
    doc = nlp(phrase)
    for token in doc:
        if token.text == target:
            pos_found = ud_to_dex[token.pos_]
            syns = get_matching_syns(token, actual_context=doc.text, pos_found=pos_found)
            try:
                return syns[:10]
            except:
                return []
    return []

print(get_syns(phrase="Faceți clic pe butonul MAC sau WIN atașat pentru a descărca fișierul font zipuit.", target="Faceți"))

[('faceți', tensor(12982.8719, dtype=torch.float64)), ('făceți-', tensor(12782.8719, dtype=torch.float64)), ('emiteți', tensor(12660.4508, dtype=torch.float64)), ('lăsați', tensor(12440.1642, dtype=torch.float64)), ('porniți', tensor(12352.1592, dtype=torch.float64)), ('produceți', tensor(12350.7451, dtype=torch.float64)), ('începeți', tensor(12153.9872, dtype=torch.float64)), ('adunați', tensor(11900.2946, dtype=torch.float64)), ('acumulați', tensor(11696.9225, dtype=torch.float64)), ('strângeți', tensor(11484.7435, dtype=torch.float64))]
